In [1]:
import pandas as pd
import numpy as np
import empyrical as ep
from datetime import datetime, timedelta

In [2]:
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [3]:
from __future__ import annotations
from pandas_market_calendars import get_calendar

In [4]:
#设计一个观察日的函数
def get_same_day_dates(start_date, n ,lock):
    dates = []
    begin_date=datetime.strptime(start_date, '%Y-%m-%d')
    for i in range(lock,n+1):
        print(i)
        current_date = begin_date
        time=i
        current_date += relativedelta(months=time)  # 将日期增加一个月
        dates.append(current_date.strftime('%Y-%m-%d'))  # 将日期对象转换为字符串格式并添加到列表中
    return dates

In [44]:
from pandas_market_calendars import get_calendar

# 获取上交所交易日历对象
sh_exchange_calendar = get_calendar('XSHG')

# 指定开始日期和结束日期
start_date = pd.to_datetime('2004-01-01')
mid_date1=pd.to_datetime('2022-09-09')
mid_date2=pd.to_datetime('2022-09-13')
end_date = pd.to_datetime('2026-12-31')

# 获取交易日日期列表
trading_days1 = sh_exchange_calendar.valid_days(start_date=start_date, end_date=mid_date1)
trading_days2 = sh_exchange_calendar.valid_days(start_date=mid_date2, end_date=end_date)

# 将交易日日期列表转换为字符串格式
trading_days_list = trading_days1.strftime('%Y-%m-%d').tolist()+trading_days2.strftime('%Y-%m-%d').tolist()
dates_to_remove = ['2022-10-06', '2022-10-07','2023-05-02','2023-05-03','2023-06-23']

for date in dates_to_remove:
    if date in trading_days_list:
        trading_days_list.remove(date)

In [45]:
#若为非交易日,则往后顺延到最近的交易日

def adjust_to_trading_days(dates, trading_days_list):
    adjusted_dates = []

    trading_days = pd.to_datetime(trading_days_list)

    for date in dates:
        dt = pd.to_datetime(date)

        if dt in trading_days:
            adjusted_dates.append(date)
        else:
            next_trading_day = trading_days[trading_days > dt].min()
            adjusted_dates.append(next_trading_day.strftime('%Y-%m-%d'))

    return adjusted_dates

In [46]:
#新撰写一个函数，作为后面计算降敲函数敲出价的辅助，date为单个日期，obs_date为日期列表，
#返回列表中早于单个日期列表的个数
def count_earlier_dates(date, obs_date):
    count = 0
    given_date = datetime.strptime(date, '%Y-%m-%d')
    
    for date_str in obs_date:
        obs = datetime.strptime(date_str, '%Y-%m-%d')
        if obs < given_date:
            count += 1
    
    return count

## 二元小雪球回测函数（含降敲设定）

In [25]:
#形成最终的结果函数
#n代表产品的期限，lock代表锁定的月度数，先以12月期，锁3个月的产品进行分析,strike代表敲出价格是期初价格的百分比,r代表敲出后的收益率,
#decline代表降敲的点数，begin_date代表开始日期，end_date代表终止日期,interrupt_date代表截断日期，roll_length代表PB,PE分位数向前滚动的天数
def snowball_dual(df_all,n,lock,strike,r,decline,df_pbpe,begin_date,end_date,interrupt_date,roll_length):
    
    df_pbpe['PB分位数']=df_pbpe['PB'].rolling(roll_length).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    df_pbpe['PE分位数']=df_pbpe['PE'].rolling(roll_length).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    df=df_all
    df.columns=['date','标的指数收盘价']
    df.index=df['date']
    df['敲出价格'] = strike*df['标的指数收盘价']
    
    #新增一列记录是否发生敲出
    df['是否敲出']=0
    
    #新增一列记录敲出或者到期或者截断的时间
    df['敲出或者到期时间']=0
    
    #新增一列记录产品敲出时指数点位
    df['敲出或到期时指数点位']=0
    
    #新增一列填入产品收益率
    df['年化收益率']=0
    
    #新增一列计算当前指数点位的历史分位数（滚动周期与PBPE计算周期相同）
    df['指数点位历史分位数']=df['标的指数收盘价'].rolling(roll_length).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    
    #新增一列判断产品是否存续
    df['产品当前是否存续']=0
    
    #新增一列计算指数近20天波动率以及波动率分位数
    df['过去20天指数波动率'] = df['标的指数收盘价'].rolling(window=20).std()
    df['过去20天波动率分位数']=df['过去20天指数波动率'].rolling(252).apply(lambda x: pd.Series(x).rank(pct=True).iloc[-1])
    
    #截取需要部分
    df=df.loc[begin_date:end_date]
    df_pbpe=df_pbpe.loc[begin_date:end_date]

    #将PB/PE的数据读出并存入
    df_part1=pd.concat([df, df_pbpe], axis=1)
    print(df_part1)
    
    #选取日截至到interrupt_date
    df_return=df_part1.loc[:interrupt_date]
    print(df_return)
    
    
    #判断是否敲出并对以上信息进行填充
    for k in range(len(df_return)):
        start_date=df_return.iloc[k,0]
        dates=get_same_day_dates(start_date, n ,lock)
        obs_date_list = adjust_to_trading_days(dates,trading_days_list)
        print(obs_date_list)
        obs_date=[date for date in obs_date_list if datetime.strptime(date, "%Y-%m-%d") <= datetime.strptime(end_date, "%Y-%m-%d")]
      
        if len(obs_date_list)>len(obs_date):
            df_return.iloc[k,8]= 1
          
        #对存在降敲的情况进行处理（即对敲出水平辅助列进行赋值和判断）
        
        #对应每个观察日生成敲出水平
        strike_level=[]
        for j in range(len(obs_date)):
            strike_level.append((strike-j*decline)*df_part1.iloc[k,1])
        
        #提取观察日对应的标的指数点位
        price_obs_date= df_part1.loc[obs_date,'标的指数收盘价']
      
        #找到敲出的时间点
        
        position = None
        for i, price in enumerate(price_obs_date):
            if price > strike_level[i]:
                position = i
                df_return.iloc[k,3]=1
                #如果发生敲出，记录产品收益率
                df_return.iloc[k,6]=r
                break

        if position is None:
            position = len(price_obs_date) - 1   
    
        #记录敲出或者到期的时间
        df_return.iloc[k,4]=obs_date[position]
        
        #记录敲出或到期时指数点位
        df_return.iloc[k,5]=df_part1.loc[obs_date[position],'标的指数收盘价']
    
    
    # 计算两个日期之间的差距
    df_return['date'] = pd.to_datetime(df_return['date'])
    df_return['敲出或者到期时间'] = pd.to_datetime(df_return['敲出或者到期时间'])
    df_return['产品存续天数'] = (df_return['敲出或者到期时间'] - df_return['date']).dt.days
    df_return['存续月数']=(df_return['产品存续天数']/30).round()

    return df_return

## 用wind接口提取数据

In [26]:
#用wind提取数据
#导入库
from WindPy import w
import pandas as pd
w.start()
#数据提取
wdata1=w.wsd("000852.SH", "close", "2013-01-01", "2023-07-28", "")
df=pd.DataFrame(wdata1.Data,columns=wdata1.Times,index=wdata1.Codes).T
df['date']=df.index
df['标的指数收盘价']=df.iloc[:,0]
df=df[['date','标的指数收盘价']]
#提取PB和PE的数据
wdata2=w.wsd("000852.SH", "pe_ttm,pb_lf", "2013-01-01", "2023-07-28", "")
df_pbpe1=pd.DataFrame(wdata2.Times)
df_pbpe2=pd.DataFrame(wdata2.Data).T
df_pbpe=pd.concat([df_pbpe1, df_pbpe2], axis=1)
df_pbpe.columns=['交易日期','PE','PB']
w.close()
#将读取的数据存入文件以防止多次读取浪费提取次数
df.to_excel('df.xlsx',index=False)
df_pbpe.to_excel('df_pbpe.xlsx',index=False)

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2020 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [47]:
#在使用wind读取并存入数据后，将数据读出，以便之后运算
df=pd.read_excel('df.xlsx')
df_pbpe=pd.read_excel('df_pbpe.xlsx')
df['date']=df['date'].dt.strftime('%Y-%m-%d')
df_pbpe['交易日期']=df_pbpe['交易日期'].dt.strftime('%Y-%m-%d')
df_pbpe.index=df_pbpe['交易日期']
df_pbpe=df_pbpe[['PE','PB']]

## 读入数据，调入函数并进行运算

In [48]:
df_return=snowball_dual(df,24,3,1.03,0.10,0,df_pbpe,"2016-01-01","2023-07-14","2023-04-14",252)
df_return.head(10)

                  date    标的指数收盘价          敲出价格  是否敲出  敲出或者到期时间  敲出或到期时指数点位  \
2016-01-04  2016-01-04  9694.8363   9985.681389     0         0           0   
2016-01-05  2016-01-05  9452.0406   9735.601818     0         0           0   
2016-01-06  2016-01-06  9709.5692  10000.856276     0         0           0   
2016-01-07  2016-01-07  8856.4914   9122.186142     0         0           0   
2016-01-08  2016-01-08  8876.2526   9142.540178     0         0           0   
...                ...        ...           ...   ...       ...         ...   
2023-07-10  2023-07-10  6526.8519   6722.657457     0         0           0   
2023-07-11  2023-07-11  6579.3695   6776.750585     0         0           0   
2023-07-12  2023-07-12  6482.9867   6677.476301     0         0           0   
2023-07-13  2023-07-13  6548.3129   6744.762287     0         0           0   
2023-07-14  2023-07-14  6544.4593   6740.793079     0         0           0   

            年化收益率  指数点位历史分位数  产品当前是否存续  过去20天指数波动率 

E:\myresource\apps\anaconda\anaconda3\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)



4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2016-04-20', '2016-05-20', '2016-06-20', '2016-07-20', '2016-08-22', '2016-09-20', '2016-10-20', '2016-11-21', '2016-12-20', '2017-01-20', '2017-02-20', '2017-03-20', '2017-04-20', '2017-05-22', '2017-06-20', '2017-07-20', '2017-08-21', '2017-09-20', '2017-10-20', '2017-11-20', '2017-12-20', '2018-01-22']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2016-04-21', '2016-05-23', '2016-06-21', '2016-07-21', '2016-08-22', '2016-09-21', '2016-10-21', '2016-11-21', '2016-12-21', '2017-01-23', '2017-02-21', '2017-03-21', '2017-04-21', '2017-05-22', '2017-06-21', '2017-07-21', '2017-08-21', '2017-09-21', '2017-10-23', '2017-11-21', '2017-12-21', '2018-01-22']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2016-04-22', '2016-05-23', '2016-06-22', '2016-07-22', '2016-08-22', '2016-09-22', '2016-10-24', '2016-11-22', '2016-12-22', '2017-01-23', '2017-02-22', '2017-03-22', '2017-04-24', '2017-05-22', '2017-06-

['2016-06-06', '2016-07-04', '2016-08-04', '2016-09-05', '2016-10-10', '2016-11-04', '2016-12-05', '2017-01-04', '2017-02-06', '2017-03-06', '2017-04-05', '2017-05-04', '2017-06-05', '2017-07-04', '2017-08-04', '2017-09-04', '2017-10-09', '2017-11-06', '2017-12-04', '2018-01-04', '2018-02-05', '2018-03-05']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2016-06-07', '2016-07-07', '2016-08-08', '2016-09-07', '2016-10-10', '2016-11-07', '2016-12-07', '2017-01-09', '2017-02-07', '2017-03-07', '2017-04-07', '2017-05-08', '2017-06-07', '2017-07-07', '2017-08-07', '2017-09-07', '2017-10-09', '2017-11-07', '2017-12-07', '2018-01-08', '2018-02-07', '2018-03-07']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2016-06-08', '2016-07-08', '2016-08-08', '2016-09-08', '2016-10-10', '2016-11-08', '2016-12-08', '2017-01-09', '2017-02-08', '2017-03-08', '2017-04-10', '2017-05-08', '2017-06-08', '2017-07-10', '2017-08-08', '2017-09-08', '2017-10-09', '2017-11-08', '2017-12-08

['2016-07-08', '2016-08-08', '2016-09-08', '2016-10-10', '2016-11-08', '2016-12-08', '2017-01-09', '2017-02-08', '2017-03-08', '2017-04-10', '2017-05-08', '2017-06-08', '2017-07-10', '2017-08-08', '2017-09-08', '2017-10-09', '2017-11-08', '2017-12-08', '2018-01-08', '2018-02-08', '2018-03-08', '2018-04-09']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2016-07-11', '2016-08-11', '2016-09-12', '2016-10-11', '2016-11-11', '2016-12-12', '2017-01-11', '2017-02-13', '2017-03-13', '2017-04-11', '2017-05-11', '2017-06-12', '2017-07-11', '2017-08-11', '2017-09-11', '2017-10-11', '2017-11-13', '2017-12-11', '2018-01-11', '2018-02-12', '2018-03-12', '2018-04-11']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2016-07-12', '2016-08-12', '2016-09-12', '2016-10-12', '2016-11-14', '2016-12-12', '2017-01-12', '2017-02-13', '2017-03-13', '2017-04-12', '2017-05-12', '2017-06-12', '2017-07-12', '2017-08-14', '2017-09-12', '2017-10-12', '2017-11-13', '2017-12-12', '2018-01-12

...

3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2023-01-16', '2023-02-14', '2023-03-14', '2023-04-14', '2023-05-15', '2023-06-14', '2023-07-14', '2023-08-14', '2023-09-14', '2023-10-16', '2023-11-14', '2023-12-14', '2024-01-15', '2024-02-16', '2024-03-14', '2024-04-15', '2024-05-14', '2024-06-14', '2024-07-15', '2024-08-14', '2024-09-16', '2024-10-14']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2023-01-17', '2023-02-17', '2023-03-17', '2023-04-17', '2023-05-17', '2023-06-19', '2023-07-17', '2023-08-17', '2023-09-18', '2023-10-17', '2023-11-17', '2023-12-18', '2024-01-17', '2024-02-19', '2024-03-18', '2024-04-17', '2024-05-17', '2024-06-17', '2024-07-17', '2024-08-19', '2024-09-18', '2024-10-17']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2023-01-18', '2023-02-20', '2023-03-20', '2023-04-18', '2023-05-18', '2023-06-19', '2023-07-18', '2023-08-18', '2023-09-18', '2023-10-18', '2023-11-20', '2023-12-18', '2024-01-18', '2024-02-19', '2024-03

13
14
15
16
17
18
19
20
21
22
23
24
['2023-03-14', '2023-04-14', '2023-05-15', '2023-06-14', '2023-07-14', '2023-08-14', '2023-09-14', '2023-10-16', '2023-11-14', '2023-12-14', '2024-01-15', '2024-02-16', '2024-03-14', '2024-04-15', '2024-05-14', '2024-06-14', '2024-07-15', '2024-08-14', '2024-09-16', '2024-10-14', '2024-11-14', '2024-12-16']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2023-03-15', '2023-04-17', '2023-05-15', '2023-06-15', '2023-07-17', '2023-08-15', '2023-09-15', '2023-10-16', '2023-11-15', '2023-12-15', '2024-01-15', '2024-02-16', '2024-03-15', '2024-04-15', '2024-05-15', '2024-06-17', '2024-07-15', '2024-08-15', '2024-09-16', '2024-10-15', '2024-11-15', '2024-12-16']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2023-03-16', '2023-04-17', '2023-05-16', '2023-06-16', '2023-07-17', '2023-08-16', '2023-09-18', '2023-10-16', '2023-11-16', '2023-12-18', '2024-01-16', '2024-02-16', '2024-03-18', '2024-04-16', '2024-05-16', '2024-06-17', '20

23
24
['2023-05-22', '2023-06-21', '2023-07-21', '2023-08-21', '2023-09-21', '2023-10-23', '2023-11-21', '2023-12-21', '2024-01-22', '2024-02-21', '2024-03-21', '2024-04-22', '2024-05-21', '2024-06-21', '2024-07-22', '2024-08-21', '2024-09-23', '2024-10-21', '2024-11-21', '2024-12-23', '2025-01-21', '2025-02-21']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2023-05-22', '2023-06-26', '2023-07-24', '2023-08-22', '2023-09-22', '2023-10-23', '2023-11-22', '2023-12-22', '2024-01-22', '2024-02-22', '2024-03-22', '2024-04-22', '2024-05-22', '2024-06-24', '2024-07-22', '2024-08-22', '2024-09-23', '2024-10-22', '2024-11-22', '2024-12-23', '2025-01-22', '2025-02-24']
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
['2023-05-23', '2023-06-26', '2023-07-24', '2023-08-23', '2023-09-25', '2023-10-23', '2023-11-23', '2023-12-25', '2024-01-23', '2024-02-23', '2024-03-25', '2024-04-23', '2024-05-23', '2024-06-24', '2024-07-23', '2024-08-23', '2024-09-23', '2024-10-23', '2024

E:\myresource\apps\anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
E:\myresource\apps\anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
E:\myresource\apps\anaconda\anaconda3\lib\site-packages\ipykernel_launcher.py:93: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docume

,date,标的指数收盘价,敲出价格,是否敲出,敲出或者到期时间,敲出或到期时指数点位,年化收益率,指数点位历史分位数,产品当前是否存续,过去20天指数波动率,过去20天波动率分位数,PE,PB,PB分位数,PE分位数,产品存续天数,存续月数
2016-01-04,2016-01-04,9694.8363,9985.681389,0,2018-01-04,7174.8650,0.0,0.571429,0,332.586530,0.345238,90.875000,4.6496,0.448413,0.619048,731,24.0
2016-01-05,2016-01-05,9452.0406,9735.601818,0,2018-01-05,7176.8679,0.0,0.531746,0,394.549866,0.416667,88.533997,4.5254,0.396825,0.559524,731,24.0
2016-01-06,2016-01-06,9709.5692,10000.856276,0,2018-01-08,7188.2906,0.0,0.571429,0,416.141196,0.440476,90.983398,4.6474,0.440476,0.623016,733,24.0
2016-01-07,2016-01-07,8856.4914,9122.186142,0,2018-01-08,7188.2906,0.0,0.396825,0,531.026725,0.607143,83.071701,4.2356,0.313492,0.456349,732,24.0
2016-01-08,2016-01-08,8876.2526,9142.540178,0,2018-01-08,7188.2906,0.0,0.400794,0,616.996279,0.706349,83.354401,4.2460,0.321429,0.464286,731,24.0
2016-01-11,2016-01-11,8205.1220,8451.275660,1,2016-04-11,8677.4091,0.1,0.309524,0,765.054996,0.773810,77.095299,3.9242,0.202381,0.345238,91,3.0
2016-01-12,2016-01-12,8187.9598,8433.598594,1,2016-04-12,8608.3738,0.1,0.305556,0,881.920886,0.797619,77.058998,3.9139,0.190476,0.341270,91,3.0
2016-01-13,2016-01-13,7836.1229,8071.206587,1,2016-04-13,8729.8406,0.1,0.242063,0,1008.882225,0.833333,73.821800,3.7476,0.107143,0.297619,91,3.0
2016-01-14,2016-01-14,8157.7372,8402.469316,1,2016-04-14,8837.0181,0.1,0.297619,0,1076.958085,0.857143,76.890198,3.9034,0.186508,0.329365,91,3.0
2016-01-15,2016-01-15,7868.1676,8104.212628,1,2016-04-15,8841.0945,0.1,0.250000,0,1140.338725,0.869048,74.167900,3.7629,0.119048,0.293651,91,3.0


## 保存数据

In [49]:
df_return.to_excel('二元小雪球回测表.xlsx',index=False)

### 根据最终结果进行胜率分析

In [50]:
#设计一个函数计算全样本胜率
def whole_win(data):
    #全样本个数
    num_total=len(data)
    #定义条件
    #1.存续，2.未敲出
    condition1=data['产品当前是否存续']==1
    condition2=data['是否敲出']==0
    #print(condition1&condition2)
    #存续且未敲出个数
    num1=data[condition1 & condition2].shape[0]
    #已敲出或到期个数
    num2=num_total-num1
    #已敲出个数
    num3=num_total-data[condition2].shape[0]
    #全样本敲出胜率
    wrate=num3/num_total
    re={'全样本个数':[num_total],'存续且未敲出个数':[num1],'已敲出或到期个数':[num2],'已敲出个数':[num3],'全样本敲出胜率':[wrate]}
    re=pd.DataFrame(re)
    return re

In [51]:
#显示全样本胜率
import pandas as pd
import numpy as np
data=pd.read_excel('二元小雪球回测表.xlsx')
whole_win(data)

,全样本个数,存续且未敲出个数,已敲出或到期个数,已敲出个数,全样本敲出胜率
0,1771,215,1556,1166,0.658385


In [52]:
#生成一个PB-PE-20天波动率分位数随敲出胜率变化的函数，step表示步长，upper代表上界,lower代表下界,kind代表想要筛选的是PB还是PE的区间,新增还可以代表过去20天
#指数波动率
def ana_pape(step,upper,lower,data,kind):
    upperline=upper+step
    df = pd.DataFrame({'分位数': [round(x, 2) for x in list(np.arange(lower, upperline, step))]})
    #生成一列储存胜率
    df['敲出胜率']=0
    for i in range(len(df)):
        #计算满足条件的总样本
        condition1=data['%s分位数'%kind]<=df.iloc[i,0]
        num1=data[condition1].shape[0]
        #求和的话，就写成：total = df.loc[condition1 & condition2, 'C'].sum()
        #计算满足条件且敲出的样本
        condition2=data['是否敲出']==1
        num2=data[condition1 & condition2].shape[0]
        #计算胜率并存入
        df.iloc[i,1]=num2/num1
    #df=df.style.format("{:.2%}")
    return df

In [63]:
pbpe

,分位数,过去20天波动率,PE,PB
0,0.01,0.604167,0.531646,0.759036
1,0.02,0.639535,0.533597,0.724832
2,0.03,0.620690,0.537217,0.713483
3,0.04,0.605634,0.521277,0.704545
4,0.05,0.585987,0.524876,0.685039
...,...,...,...,...
95,0.96,0.653731,0.641711,0.650296
96,0.97,0.654706,0.643237,0.651666
97,0.98,0.655594,0.644497,0.653085
98,0.99,0.655530,0.646406,0.656286


In [60]:
#读取数据并采用函数计算结果和绘图
import pandas as pd
import numpy as np
data=pd.read_excel('二元小雪球回测表.xlsx')
pvol=ana_pape(0.01,1.00,0.01,data,'过去20天波动率')
pe=ana_pape(0.01,1.00,0.01,data,'PE')
pb=ana_pape(0.01,1.00,0.01,data,'PB')
pbpe=pd.merge(pvol, pe, on='分位数').merge(pb, on='分位数')
pbpe.columns=['分位数','过去20天波动率','PE','PB']

In [64]:
#绘制二维折线图
import pyecharts.options as opts
from pyecharts.charts import Line
from pyecharts.faker import Faker

c = (
    Line()
    .add_xaxis(xaxis_data=np.sort(pbpe['分位数'].tolist()))
    .add_yaxis("PB分位数敲出胜率", y_axis=pbpe['PB'].tolist())
    .add_yaxis("PE分位数敲出胜率", y_axis=pbpe['PE'].tolist())
    .add_yaxis("过去20天波动率分位数敲出胜率", y_axis=pbpe['过去20天波动率'].tolist())
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))  # 不显示标签
    .set_global_opts(title_opts=opts.TitleOpts(title="分位数敲出胜率展示图"))
    .render("分位数敲出胜率展示图.html")
)